
# Data Preparation

## Imports

In [1]:
from os import listdir
import os.path
from os.path import isfile, join
from osgeo import gdal
import numpy as np

In [2]:
# for coloured print statements
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

In [3]:
path_labels = os.getcwd() + "/1_labeled_data/tif/"
path_rdg = "/home/jovyan/work/satellite_data/"
label_tif = path_labels + "22-01-05.TIF"

In [4]:
def print_files(path = os.getcwd()):
    onlyfiles = [f for f in listdir(path) if isfile(join(path, f))]
    print(path, onlyfiles)
print_files(path_labels)

/home/jovyan/work/saved_data/Semantic_Segmentation_of_Tree_Stock/1_labeled_data/tif/ ['22-01-05-uploaded.tif', '22-01-05.TIF', 'check-labels-jp2-capizalized-22-01-05.TIF', 'labels-jp2-tif-22.01-05.tif', '22-01-05-test-3-without-tif.tif', 'now.tif', '22-01-30.tif', '22-01-06-with-tif.tif']


### 0.1 Data Inspection

In [5]:
# Import Image with GDAL
dataset = gdal.Open(path_rdg + "2629BD_2018.tif")

In [6]:
# General Dataset Information
def inspect_tif(dataset):
    print("1. General TIF Data:")
    # print("Metadata:",7 dataset.GetMetadata())
    # print("Type Image: ", type(dataset))
    print("Projection: ", dataset.GetProjection())
    print("Size is {} x {} x {}".format(dataset.RasterXSize,
                                    dataset.RasterYSize,
                                    dataset.RasterCount))
    # print("Driver: {}/{}".format(dataset.GetDriver().ShortName,
    #                         dataset.GetDriver().LongName))
    # geotransform = dataset.GetGeoTransform()
    # if geotransform:
    #     print("Origin = ({}, {})".format(geotransform[0], geotransform[3]))
    #     print("Pixel Size = ({}, {})".format(geotransform[1], geotransform[5]))

    # Raster Band
    band = dataset.GetRasterBand(1)
    print("2. Raster Band:")
    # print("Band Type={}".format(gdal.GetDataTypeName(band.DataType)))

    min = band.GetMinimum()
    max = band.GetMaximum()
    if not min or not max:
        (min,max) = band.ComputeRasterMinMax(True)
    print("Min={:.3f}, Max={:.3f}".format(min,max))

    if band.GetOverviewCount() > 0:
        print("Band has {} overviews".format(band.GetOverviewCount()))

    # if band.GetRasterColorTable():
    #     print("Band has a color table with {} entries".format(band.GetRasterColorTable().GetCount()))
        
    size_x_px = dataset.RasterXSize
    size_y_px = dataset.RasterYSize
    pixel_size = (dataset.GetGeoTransform()[1], dataset.GetGeoTransform()[1])
    size_x = size_x_px * pixel_size[0]
    size_y = size_y_px * pixel_size[1]
    print("""Image Size: 
        # Pixels in Image = ({}, {})
        Size of one Pixel in cm = {}
        Size Covered by Image / Image Size = ({}m, {}m)""".format(size_x_px, size_y_px, 
                                                                                    tuple([100*x for x in pixel_size]),
                                                                                    size_x, size_y)
    )
inspect_tif(dataset)

1. General TIF Data:
Projection:  GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]
Size is 108224 x 107798 x 3
2. Raster Band:
Min=0.000, Max=255.000
Band has 8 overviews
Image Size: 
        # Pixels in Image = (108224, 107798)
        Size of one Pixel in cm = (0.00023728143101288307, 0.00023728143101288307)
        Size Covered by Image / Image Size = (0.2567954558993826m, 0.2557846370032677m)


In [7]:
some_dataset = gdal.Open(path_rdg + "2627DC_2018.tif")
inspect_tif(some_dataset)

1. General TIF Data:
Projection:  GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]
Size is 107608 x 107249 x 3
2. Raster Band:
Min=0.000, Max=240.000
Band has 8 overviews
Image Size: 
        # Pixels in Image = (107608, 107249)
        Size of one Pixel in cm = (0.00023770193857792506, 0.00023770193857792506)
        Size Covered by Image / Image Size = (0.2557863020649336m, 0.25493295210543887m)


In [8]:
# optional conversion to np array
# channel = np.array(dataset.GetRasterBand(1).ReadAsArray())
# print(channel.shape)

In [9]:
# Read the raster band as separate variable
#band = dataset.GetRasterBand(1)

# Check type of the variable 'band'
#type(band)

# Data type of the values
#gdal.GetDataTypeName(band.DataType)

In [10]:
import matplotlib.pyplot as plt
#print(dataset.RasterCount)
#print(labels.RasterCount, labels.RasterXSize, labels.RasterYSize)
print(dataset.RasterCount, dataset.RasterXSize, dataset.RasterYSize)

3 108224 107798


In [11]:
# band1 = labels.GetRasterBand(1) # Red channel
# band2 = labels.GetRasterBand(2) # Green channel
# band3 = labels.GetRasterBand(3) # Blue channel

In [12]:
#labels.GetRasterBand()

### 0.2 Load Labels

In [13]:
"""
Idea Training Process:
1. Load training set (labels with corresponding input TIFs)
2. Create cutouts of labeled training data
3. Train model on data
4. Evaluate model

Idea Prediction Process:
5. Create cutouts of unlabeled data
6. Predict cutouts

Function Overview
Training: load labels -> create_cutouts(label_file) -> fit(unlabeled_cutouts, labeled_cutouts) -> evaluate(?)
Prediction: load dats -> create_cutouts(label_file) -> predict(unlabeled_cutouts)
"""

'\nIdea Training Process:\n1. Load training set (labels with corresponding input TIFs)\n2. Create cutouts of labeled training data\n3. Train model on data\n4. Evaluate model\n\nIdea Prediction Process:\n5. Create cutouts of unlabeled data\n6. Predict cutouts\n\nFunction Overview\nTraining: load labels -> create_cutouts(label_file) -> fit(unlabeled_cutouts, labeled_cutouts) -> evaluate(?)\nPrediction: load dats -> create_cutouts(label_file) -> predict(unlabeled_cutouts)\n'

In [14]:
def create_cutouts(path_label, label_file):
    # upper/lower (u/l) left/right (l/r) coordinates 
    ulx, xres, xskew, uly, yskew, yres  = label_file.GetGeoTransform()
    width, height = label_file.RasterXSize, label_file.RasterYSize
    lrx = ulx + width * xres
    lry = uly + height * yres
    # corner & cener coordinates can be checked in the terminal with gdalinfo '/home/jovyan/work/satellite_data/FILE.tif'
    # determine cutout window of size: 512x512 px
    cutout_size = 512
    crop_ulx = ulx
    crop_uly = uly
    crop_lrx = ulx + cutout_size * xres
    crop_lry = uly + cutout_size * yres
    cropped_window = (crop_ulx,crop_uly,crop_lrx,crop_lry)
    print("cropped window:", cropped_window)
    #export cropped raster
    print("label_file:", path_label)
    cutout_path = os.getcwd() + "/2_cutouts/" + path_label
    gdal.Translate(cutout_path, label_file, projWin = cropped_window) #example_tif is the tif file

In [16]:
def load_tif(label_files):
    # loop through array of file names
    for path_label in label_files:
        print(f"{bcolors.OKBLUE}{path_label}{bcolors.ENDC}")
        path_label_file = path_labels + path_label
        # check if filepath exists
        if not os.path.isfile(path_label_file):
            print("Filepath does not exist") 
            continue
        # open file
        label_file = gdal.Open(path_label_file)
        # check if gdal is able to open file
        if label_file:
            print(f"{bcolors.OKGREEN}0 Success:{bcolors.ENDC}" + path_label_file)
            inspect_tif(label_file)
            # create cutouts
            create_cutouts(path_label, label_file)
            continue
        print(f"{bcolors.FAIL}1 Failure:{bcolors.ENDC}" + path_label_file) #"File cannot be loaded." #"Filepath exists: ", os.path.isfile(tif_filepath), 

22-01-30.tif
0 Success:/home/jovyan/work/saved_data/Semantic_Segmentation_of_Tree_Stock/1_labeled_data/tif/22-01-30.tif
1. General TIF Data:
Projection:  GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]
Size is 4112 x 11377 x 4
2. Raster Band:
Min=80.000, Max=255.000
Image Size: 
        # Pixels in Image = (4112, 11377)
        Size of one Pixel in cm = (0.0015679484435797747, 0.0015679484435797747)
        Size Covered by Image / Image Size = (0.06447404000000034m, 0.17838549442607096m)
cropped window: (29.748224553, -26.248672807786964, 29.756252449031127, -26.25670070381809)
label_file: 22-01-30.tif


In [ ]:
label_files = ["22-01-30.tif"] #, "now.tif"
load_tiif(label_files)

# Backlog

In [17]:
#check if all (label) files in a directory are loadable
def check_all_labels(directory = os.fsencode(path_labels)):
    print("Labels")    
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename.endswith(".tif") or filename.endswith(".TIF") or filename.endswith(".tiff"): 
            print(filename)
            load_tif(path_labels + filenyame)
            print("")
            continue
        else:
            continue
#check_all_labels(os.fsencode(path_labels))

### 0.3 Preview labels (TIF)

In [20]:
import rasterio
from rasterio.plot import show
# label = rasterio.open(tif_filepath)
# show(label)

In [21]:
# No. of Bands, Image resolution (X, Y), CRS (Coordinate Reference System)
# print(label.count, label.height, label.width, label.crs)

In [22]:
def preview_tif(tif_filepath):
    if os.path.isfile(tif_filepath):
        img = rasterio.open(tif_filepath)
        show(img)
    else: print("File does not exist.")

In [23]:
example_tif = path_rdg + "2629BD_2018.tif"
# preview_tif(example_tif)

### 0.4 Crop TIF

In [ ]:
load_tif(cropped_raster)

In [ ]:
preview_tif(cropped_raster)

In [ ]:
# close tif
#dataset = None